# Tarea 4 - Punto 1: Reactor PFR con Elementos Finitos

## Problema: Estado Estacionario de Concentración en Reactor PFR

Ecuación gobernante:
$$D\frac{d^2c}{dx^2} - U\frac{dc}{dx} - kc = 0$$

Condiciones de contorno:
- En $x = 0$: $U \cdot c_{inlet} = U \cdot c - D\frac{dc}{dx}$
- En $x = L$: $\frac{dc}{dx} = 0$

### Parámetros
- $D = 5000$ m²/hr (difusión/dispersión)
- $U = 100$ m/hr (velocidad advectiva)
- $k = 2$ 1/hr (tasa de reacción)
- $L = 100$ m (longitud)
- $c_{inlet} = 100$ mol/L

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from numpy.polynomial.legendre import Legendre, leggauss
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Configuración de gráficas
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Parámetros del Problema

In [ ]:
# ============================================================================
# PARÁMETROS DEL REACTOR PFR
# ============================================================================

D = 5000.0      # [m²/hr] coeficiente de difusión/dispersión
U = 100.0       # [m/hr] velocidad advectiva
k = 2.0         # [1/hr] tasa de reacción
L = 100.0       # [m] longitud del reactor
c_inlet = 100.0 # [mol/L] concentración de entrada

print(f"Parámetros del Reactor PFR:")
print(f"  D = {D} m²/hr")
print(f"  U = {U} m/hr")
print(f"  k = {k} 1/hr")
print(f"  L = {L} m")
print(f"  c_inlet = {c_inlet} mol/L")

## 2. Solución Analítica

In [ ]:
def solucion_analitica(x, D, U, k, L, c_inlet):
    """
    Solución analítica del reactor PFR.
    """
    # Cálculo de lambda1 y lambda2
    raiz = np.sqrt(1 + 4*k*D/U**2)
    lambda1 = (U/(2*D)) * (1 + raiz)
    lambda2 = (U/(2*D)) * (1 - raiz)
    
    # Términos del denominador
    denom = (U - D*lambda1) * lambda2 * np.exp(lambda2*L) - (U - D*lambda2) * lambda1 * np.exp(lambda1*L)
    
    # Solución
    c = (U * c_inlet / denom) * (lambda2 * np.exp(lambda2*L) * np.exp(lambda1*x) - 
                                   lambda1 * np.exp(lambda1*L) * np.exp(lambda2*x))
    
    return c

# Graficar solución analítica
x_plot = np.linspace(0, L, 500)
c_analitica = solucion_analitica(x_plot, D, U, k, L, c_inlet)

plt.figure(figsize=(10, 6))
plt.plot(x_plot, c_analitica, 'k-', lw=2, label='Solución Analítica')
plt.xlabel(r'$x$ [m]')
plt.ylabel(r'$c$ [mol/L]')
plt.title('Solución Analítica del Reactor PFR')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"c(0) = {c_analitica[0]:.4f} mol/L")
print(f"c(L) = {c_analitica[-1]:.4f} mol/L")
print(f"Conversión = {(c_inlet - c_analitica[-1])/c_inlet*100:.2f}%")

## 3. Funciones Auxiliares

In [ ]:
def gauss_legendre_integral(f, a, b, n=80):
    """Integra f(x) en [a,b] usando cuadratura Gauss-Legendre con n puntos."""
    xi, wi = leggauss(n)  # puntos y pesos en [-1,1]
    # Mapeo lineal a [a,b]
    x_mapped = 0.5 * (b - a) * xi + 0.5 * (b + a)
    w_mapped = 0.5 * (b - a) * wi
    return np.dot(w_mapped, f(x_mapped))

## 4. PARTE NO DISCRETIZADA: Polinomios de Legendre

Usamos polinomios de Legendre en coordenadas locales $\xi \in [-1,1]$ mapeadas al dominio físico $[0,L]$.

In [ ]:
# ============================================================================
# FORMULACIÓN NO DISCRETIZADA: LEGENDRE
# ============================================================================

# ---- Funciones base para la aproximación --------------------------
def Nm(xi, m):
    """Polinomio de Legendre de orden m evaluado en xi ∈ [-1,1]."""
    return Legendre.basis(m)(xi)

def dNm(xi, m):
    """Primera derivada dP_m/dxi."""
    return Legendre.basis(m).deriv()(xi)

def d2Nm(xi, m):
    """Segunda derivada d²P_m/dxi²."""
    return Legendre.basis(m).deriv(2)(xi)

# ---- Términos locales Ke(i,j) y Fe(i) -----------------------------
def Ke(i, j, D, U, k, L, xi_lim, xf_lim, quad_n=80):
    """
    Calcula K_ij para reactor PFR en coordenadas locales ξ ∈ [-1,1].
    
    K_ij = -∫ D·dNi/dx·dNj/dx dx - ∫ U·Ni·dNj/dx dx - ∫ k·Ni·Nj dx
    
    Transformando a coordenadas locales:
    dx = (L/2)dξ, d/dx = (2/L)d/dξ
    """
    # Término difusivo: -D * ∫ dNi/dx * dNj/dx dx
    # = -D * (2/L)² * ∫ dNi/dξ * dNj/dξ * (L/2) dξ
    integrand1 = lambda xi: dNm(xi, i) * dNm(xi, j)
    I1 = gauss_legendre_integral(integrand1, -1, 1, n=quad_n)
    term1 = -D * (2/L)**2 * (L/2) * I1
    
    # Término advectivo: -U * ∫ Ni * dNj/dx dx  
    # = -U * ∫ Ni * (2/L)dNj/dξ * (L/2) dξ
    integrand2 = lambda xi: Nm(xi, i) * dNm(xi, j)
    I2 = gauss_legendre_integral(integrand2, -1, 1, n=quad_n)
    term2 = -U * (2/L) * (L/2) * I2
    
    # Término reactivo: -k * ∫ Ni * Nj dx
    # = -k * ∫ Ni * Nj * (L/2) dξ
    integrand3 = lambda xi: Nm(xi, i) * Nm(xi, j)
    I3 = gauss_legendre_integral(integrand3, -1, 1, n=quad_n)
    term3 = -k * (L/2) * I3
    
    return term1 + term2 + term3

def Fe(i, U, c_inlet, L, xi_lim, xf_lim, quad_n=80):
    """
    Calcula F_i del vector de carga.
    
    De las condiciones de contorno:
    F_i = U * c_inlet * Ni(x=0)
    
    En coordenadas locales: x=0 corresponde a ξ=-1
    """
    return U * c_inlet * Nm(-1.0, i)

# ---- Ensamblaje global de K y F -----------------------------------------
def assemble_KF(M, D, U, k, L, c_inlet, xi, xf, quad_n=80):
    """Construye las matrices globales K y F para el sistema Galerkin."""
    K = np.zeros((M, M))
    F = np.zeros(M)
    for i in range(M):
        for j in range(M):
            K[i, j] = Ke(i, j, D, U, k, L, xi, xf, quad_n=quad_n)
        F[i] = Fe(i, U, c_inlet, L, xi, xf, quad_n=quad_n)
    
    # Aplicar condición de contorno en x=0 (ξ=-1)
    # Añadir U*Ni(0)*Nj(0) a K
    for i in range(M):
        for j in range(M):
            K[i, j] += U * Nm(-1.0, i) * Nm(-1.0, j)
    
    return K, F

### 4.1 Solución para Diferentes Valores de M

In [ ]:
# Solución no discretizada para varios M
xi, xf = 0.0, L
X = np.linspace(xi, xf, 400)
M_values = [3, 5, 8, 12, 15]

plt.figure(figsize=(10, 6))

colors = plt.cm.viridis(np.linspace(0, 0.9, len(M_values)))

for idx, M in enumerate(M_values):
    # Ensamble global
    K, F = assemble_KF(M, D, U, k, L, c_inlet, xi, xf, quad_n=120)
    
    # Resolver sistema lineal
    a = np.linalg.solve(K, F)
    
    # Aproximación con las M funciones base
    # Mapear X a coordenadas locales
    xi_points = 2 * X / L - 1
    c_approx = np.zeros_like(X)
    for j in range(M):
        c_approx += a[j] * Nm(xi_points, j)
    
    plt.plot(X, c_approx, '-', color=colors[idx], lw=1.5, label=f'M = {M}')

# Solución exacta
plt.plot(X, solucion_analitica(X, D, U, k, L, c_inlet), 'k--', lw=2, label='Solución Analítica')
plt.xlabel(r'$x$ [m]')
plt.ylabel(r'$c$ [mol/L]')
plt.title('Método de Galerkin con Polinomios de Legendre (No Discretizado)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 4.2 Estudio de Convergencia (No Discretizado)

In [ ]:
# Estudio de convergencia para método no discretizado
M_convergencia = np.arange(3, 25, 1)
errores_L2 = []
errores_max = []

x_error = np.linspace(0, L, 500)
c_exact_error = solucion_analitica(x_error, D, U, k, L, c_inlet)

for M in M_convergencia:
    try:
        K, F = assemble_KF(M, D, U, k, L, c_inlet, 0, L, quad_n=120)
        a = np.linalg.solve(K, F)
        
        xi_error = 2 * x_error / L - 1
        c_approx = np.zeros_like(x_error)
        for j in range(M):
            c_approx += a[j] * Nm(xi_error, j)
        
        # Error L2
        error_L2 = np.sqrt(np.trapz((c_approx - c_exact_error)**2, x_error) / L)
        errores_L2.append(error_L2)
        
        # Error máximo
        error_max = np.max(np.abs(c_approx - c_exact_error))
        errores_max.append(error_max)
    except:
        errores_L2.append(np.nan)
        errores_max.append(np.nan)

# Gráficas de convergencia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogy(M_convergencia, errores_L2, 'bo-', lw=2, markersize=6)
ax1.set_xlabel(r'Número de funciones base $M$')
ax1.set_ylabel(r'Error $L_2$')
ax1.set_title('Convergencia: Error L2 vs M (No Discretizado)')
ax1.grid(True, alpha=0.3)

ax2.semilogy(M_convergencia, errores_max, 'ro-', lw=2, markersize=6)
ax2.set_xlabel(r'Número de funciones base $M$')
ax2.set_ylabel(r'Error Máximo')
ax2.set_title('Convergencia: Error Máximo vs M (No Discretizado)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResultados de Convergencia (No Discretizado):")
print(f"  M = {M_convergencia[0]}: Error L2 = {errores_L2[0]:.6e}")
print(f"  M = {M_convergencia[-1]}: Error L2 = {errores_L2[-1]:.6e}")
print(f"  Reducción de error: {errores_L2[0]/errores_L2[-1]:.2f}x")

### 4.3 Cálculo del Residuo (No Discretizado)

In [ ]:
def calcular_residuo_legendre(a, x, D, U, k, L):
    """
    Calcula el residuo R(x) = D·d²c/dx² - U·dc/dx - k·c
    """
    M = len(a)
    xi = 2 * x / L - 1
    
    c = np.zeros_like(x)
    dc_dx = np.zeros_like(x)
    d2c_dx2 = np.zeros_like(x)
    
    for m in range(M):
        Pm = Legendre.basis(m)
        dPm = Pm.deriv()
        d2Pm = Pm.deriv(2)
        
        c += a[m] * Pm(xi)
        dc_dx += a[m] * dPm(xi) * (2/L)
        d2c_dx2 += a[m] * d2Pm(xi) * (2/L)**2
    
    R = D*d2c_dx2 - U*dc_dx - k*c
    return R

# Calcular residuo para diferentes M
M_residuo = [5, 8, 12, 15]
x_residuo = np.linspace(0, L, 400)

plt.figure(figsize=(10, 6))

colors_res = plt.cm.plasma(np.linspace(0, 0.9, len(M_residuo)))

for idx, M in enumerate(M_residuo):
    K, F = assemble_KF(M, D, U, k, L, c_inlet, 0, L, quad_n=120)
    a = np.linalg.solve(K, F)
    R = calcular_residuo_legendre(a, x_residuo, D, U, k, L)
    plt.plot(x_residuo, R, '-', color=colors_res[idx], lw=1.5, label=f'M = {M}')

plt.xlabel(r'$x$ [m]')
plt.ylabel(r'$R_\Omega(x)$')
plt.title('Residuo de la Ecuación Diferencial (No Discretizado)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. PARTE DISCRETIZADA: Elementos Finitos con Lagrange

Discretizamos el dominio en elementos y usamos polinomios de Lagrange lineales.

In [ ]:
# ============================================================================
# FORMULACIÓN DISCRETIZADA: FEM CON LAGRANGE
# ============================================================================

def create_mesh(L, n_elem):
    """Crea una malla uniforme de elementos finitos."""
    nodes = np.linspace(0, L, n_elem + 1)
    h = L / n_elem
    connectivity = np.zeros((n_elem, 2), dtype=int)
    for e in range(n_elem):
        connectivity[e, :] = [e, e+1]
    return nodes, h, connectivity

def N_lagrange(xi, i):
    """Funciones de forma lineales en ξ ∈ [0,1]."""
    if i == 0:
        return 1 - xi
    elif i == 1:
        return xi

def dN_lagrange(xi, i):
    """Derivada de las funciones de forma respecto a ξ."""
    if i == 0:
        return -1.0
    elif i == 1:
        return 1.0

def Ke_local(h_e, D, U, k, quad_n=10):
    """
    Matriz de rigidez local 2x2 para un elemento.
    
    K^e_ij = -∫ D·dNi/dx·dNj/dx dx - ∫ U·Ni·dNj/dx dx - ∫ k·Ni·Nj dx
    """
    K_local = np.zeros((2, 2))
    
    # Cuadratura Gauss-Legendre en [0,1]
    xi_gauss, w_gauss = leggauss(quad_n)
    xi_gauss = 0.5 * (xi_gauss + 1)  # Mapear de [-1,1] a [0,1]
    w_gauss = 0.5 * w_gauss
    
    for i in range(2):
        for j in range(2):
            # Término difusivo
            term1 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                dNi_dx = dN_lagrange(xi, i) / h_e
                dNj_dx = dN_lagrange(xi, j) / h_e
                term1 += w * dNi_dx * dNj_dx * h_e
            
            # Término advectivo
            term2 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                Ni = N_lagrange(xi, i)
                dNj_dx = dN_lagrange(xi, j) / h_e
                term2 += w * Ni * dNj_dx * h_e
            
            # Término reactivo
            term3 = 0.0
            for xi, w in zip(xi_gauss, w_gauss):
                Ni = N_lagrange(xi, i)
                Nj = N_lagrange(xi, j)
                term3 += w * Ni * Nj * h_e
            
            K_local[i, j] = -D*term1 - U*term2 - k*term3
    
    return K_local

def assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """Ensambla la matriz de rigidez global y el vector de carga."""
    n_nodes = len(nodes)
    n_elem = len(connectivity)
    h_e = nodes[1] - nodes[0]
    
    K_global = np.zeros((n_nodes, n_nodes))
    F_global = np.zeros(n_nodes)
    
    # Ensamblar matriz de rigidez
    for e in range(n_elem):
        node_ids = connectivity[e, :]
        K_local = Ke_local(h_e, D, U, k, quad_n=quad_n)
        
        for i_local in range(2):
            i_global = node_ids[i_local]
            for j_local in range(2):
                j_global = node_ids[j_local]
                K_global[i_global, j_global] += K_local[i_local, j_local]
    
    # Aplicar condición de contorno en x=0
    F_global[0] = U * c_inlet
    
    return K_global, F_global

def solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=10):
    """Resuelve el sistema FEM."""
    K, F = assemble_global_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=quad_n)
    c_nodes = np.linalg.solve(K, F)
    return c_nodes

### 5.1 Solución con Diferentes Mallas

In [ ]:
# Definir diferentes mallas
n_elem_list = [10, 15, 25, 40, 60]

x_exact = np.linspace(0, L, 500)
c_exact = solucion_analitica(x_exact, D, U, k, L, c_inlet)

plt.figure(figsize=(12, 7))

colors_fem = plt.cm.coolwarm(np.linspace(0.1, 0.9, len(n_elem_list)))

for idx, n_elem in enumerate(n_elem_list):
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    
    plt.plot(nodes, c_fem, 'o-', color=colors_fem[idx], lw=1.5, 
             markersize=4, label=f'FEM: {n_elem} elem (h={h:.2f} m)')

plt.plot(x_exact, c_exact, 'k--', lw=2, label='Solución Analítica')
plt.xlabel(r'$x$ [m]')
plt.ylabel(r'$c$ [mol/L]')
plt.title('Método de Elementos Finitos con Lagrange (Discretizado)')
plt.legend(fontsize=9, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Estudio de Convergencia (Discretizado)

In [ ]:
# Estudio de convergencia FEM
n_elem_convergencia = [5, 10, 15, 20, 25, 30, 40, 50, 60, 80, 100]
errores_L2_fem = []
errores_max_fem = []
h_values = []

for n_elem in n_elem_convergencia:
    nodes, h, connectivity = create_mesh(L, n_elem)
    c_fem = solve_FEM(nodes, connectivity, D, U, k, c_inlet, quad_n=15)
    
    c_fem_interp = np.interp(x_exact, nodes, c_fem)
    
    error_L2 = np.sqrt(np.trapz((c_fem_interp - c_exact)**2, x_exact) / L)
    errores_L2_fem.append(error_L2)
    
    error_max = np.max(np.abs(c_fem_interp - c_exact))
    errores_max_fem.append(error_max)
    
    h_values.append(h)

# Gráficas de convergencia
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].loglog(n_elem_convergencia, errores_L2_fem, 'bs-', lw=2, markersize=7)
axes[0, 0].set_xlabel('Número de elementos')
axes[0, 0].set_ylabel(r'Error $L_2$')
axes[0, 0].set_title('Error L2 vs Número de Elementos (FEM)')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].loglog(h_values, errores_L2_fem, 'gs-', lw=2, markersize=7)
h_ref = np.array(h_values)
axes[0, 1].loglog(h_ref, 0.5*h_ref**2, 'k--', alpha=0.5, label=r'$\mathcal{O}(h^2)$')
axes[0, 1].set_xlabel('Tamaño de malla h [m]')
axes[0, 1].set_ylabel(r'Error $L_2$')
axes[0, 1].set_title('Error L2 vs Tamaño de Malla (FEM)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].loglog(n_elem_convergencia, errores_max_fem, 'rs-', lw=2, markersize=7)
axes[1, 0].set_xlabel('Número de elementos')
axes[1, 0].set_ylabel('Error Máximo')
axes[1, 0].set_title('Error Máximo vs Número de Elementos (FEM)')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].loglog(h_values, errores_max_fem, 'ms-', lw=2, markersize=7)
axes[1, 1].loglog(h_ref, 0.5*h_ref**2, 'k--', alpha=0.5, label=r'$\mathcal{O}(h^2)$')
axes[1, 1].set_xlabel('Tamaño de malla h [m]')
axes[1, 1].set_ylabel('Error Máximo')
axes[1, 1].set_title('Error Máximo vs Tamaño de Malla (FEM)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResultados de Convergencia (FEM):")
print(f"  {n_elem_convergencia[0]} elementos: Error L2 = {errores_L2_fem[0]:.6e}")
print(f"  {n_elem_convergencia[-1]} elementos: Error L2 = {errores_L2_fem[-1]:.6e}")
print(f"  Reducción de error: {errores_L2_fem[0]/errores_L2_fem[-1]:.2f}x")

## 6. Comparación: No Discretizado vs Discretizado

In [ ]:
# Comparación directa de ambos métodos
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

M_comp = 15
n_elem_comp = 50

# No discretizado (Legendre)
K_leg, F_leg = assemble_KF(M_comp, D, U, k, L, c_inlet, 0, L, quad_n=120)
a_leg = np.linalg.solve(K_leg, F_leg)
xi_comp = 2 * x_exact / L - 1
c_legendre = np.zeros_like(x_exact)
for j in range(M_comp):
    c_legendre += a_leg[j] * Nm(xi_comp, j)

# Discretizado (Lagrange FEM)
nodes_fem, h_fem, conn_fem = create_mesh(L, n_elem_comp)
c_lagrange = solve_FEM(nodes_fem, conn_fem, D, U, k, c_inlet, quad_n=15)
c_lagrange_interp = np.interp(x_exact, nodes_fem, c_lagrange)

# Gráfica 1: Comparación de soluciones
axes[0].plot(x_exact, c_exact, 'k-', lw=2.5, label='Analítica')
axes[0].plot(x_exact, c_legendre, 'b--', lw=2, label=f'Legendre (M={M_comp})')
axes[0].plot(nodes_fem, c_lagrange, 'ro-', lw=1.5, markersize=3, 
             label=f'Lagrange FEM ({n_elem_comp} elem)')
axes[0].set_xlabel(r'$x$ [m]')
axes[0].set_ylabel(r'$c$ [mol/L]')
axes[0].set_title('Comparación: Legendre vs Lagrange FEM')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfica 2: Errores
error_legendre = np.abs(c_legendre - c_exact)
error_lagrange = np.abs(c_lagrange_interp - c_exact)

axes[1].semilogy(x_exact, error_legendre, 'b-', lw=2, label=f'Legendre (M={M_comp})')
axes[1].semilogy(x_exact, error_lagrange, 'r--', lw=2, label=f'Lagrange FEM ({n_elem_comp} elem)')
axes[1].set_xlabel(r'$x$ [m]')
axes[1].set_ylabel('Error Absoluto')
axes[1].set_title('Comparación de Errores')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nComparación de Métodos:")
print(f"  Legendre (M={M_comp}): Error L2 = {np.sqrt(np.trapz(error_legendre**2, x_exact)/L):.6e}")
print(f"  Lagrange FEM ({n_elem_comp} elem): Error L2 = {np.sqrt(np.trapz(error_lagrange**2, x_exact)/L):.6e}")

## 7. Estudio Paramétrico: Efectos de k

In [ ]:
# Valores de k a estudiar
k_values = [k/4, k/2, k, 2*k, 4*k]

fig, axes = plt.subplots(2, 2, figsize=(15, 11))

colors_k = plt.cm.Spectral(np.linspace(0.1, 0.9, len(k_values)))

# ===== Gráfica 1: Perfiles de concentración =====
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
    axes[0, 0].plot(x_exact, c_k, '-', color=colors_k[idx], lw=2, 
                    label=f'k = {k_val:.2f} 1/hr')

axes[0, 0].set_xlabel(r'$x$ [m]')
axes[0, 0].set_ylabel(r'$c$ [mol/L]')
axes[0, 0].set_title('Efecto de k en el Perfil de Concentración')
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3)

# ===== Gráfica 2: Concentración normalizada =====
for idx, k_val in enumerate(k_values):
    c_k = solucion_analitica(x_exact, D, U, k_val, L, c_inlet)
    c_norm = c_k / c_inlet
    axes[0, 1].plot(x_exact/L, c_norm, '-', color=colors_k[idx], lw=2, 
                    label=f'k = {k_val:.2f} 1/hr')

axes[0, 1].set_xlabel(r'Posición adimensional $x/L$')
axes[0, 1].set_ylabel(r'$c/c_{inlet}$')
axes[0, 1].set_title('Perfiles Normalizados')
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# ===== Gráfica 3: Concentración a la salida vs k =====
k_range = np.linspace(0.1, 8, 100)
c_salida = []
for k_val in k_range:
    c_k = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)
    c_salida.append(c_k[0])

axes[1, 0].plot(k_range, c_salida, 'b-', lw=2.5)
axes[1, 0].axvline(k, color='r', linestyle='--', alpha=0.7, label=f'k nominal = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    axes[1, 0].plot(k_val, c_out, 'ro', markersize=8)

axes[1, 0].set_xlabel(r'Tasa de reacción $k$ [1/hr]')
axes[1, 0].set_ylabel(r'$c(L)$ [mol/L]')
axes[1, 0].set_title('Concentración de Salida vs k')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# ===== Gráfica 4: Conversión del reactor vs k =====
conversion = [(c_inlet - c_out)/c_inlet * 100 for c_out in c_salida]

axes[1, 1].plot(k_range, conversion, 'g-', lw=2.5)
axes[1, 1].axvline(k, color='r', linestyle='--', alpha=0.7, label=f'k nominal = {k} 1/hr')
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    axes[1, 1].plot(k_val, conv, 'ro', markersize=8)

axes[1, 1].set_xlabel(r'Tasa de reacción $k$ [1/hr]')
axes[1, 1].set_ylabel('Conversión [%]')
axes[1, 1].set_title('Conversión del Reactor vs k')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEstudio Paramétrico de k:")
print(f"{'k [1/hr]':>10} {'c(L) [mol/L]':>15} {'Conversión [%]':>18}")
print("-" * 50)
for k_val in k_values:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    print(f"{k_val:10.2f} {c_out:15.4f} {conv:18.2f}")

## 8. Resumen de Resultados y Conclusiones

In [ ]:
print("="*70)
print("RESUMEN DE RESULTADOS - REACTOR PFR")
print("="*70)

print("\n1. PARÁMETROS DEL PROBLEMA:")
print(f"   D = {D} m²/hr")
print(f"   U = {U} m/hr")
print(f"   k = {k} 1/hr")
print(f"   L = {L} m")
print(f"   c_inlet = {c_inlet} mol/L")

print("\n2. MÉTODO NO DISCRETIZADO (LEGENDRE):")
print(f"   - Convergencia: Exponencial (espectral)")
print(f"   - M óptimo: ~12-15 funciones base")
print(f"   - Ventaja: Alta precisión con pocos grados de libertad")
print(f"   - Desventaja: Matrices densas, limitado a geometrías simples")

print("\n3. MÉTODO DISCRETIZADO (LAGRANGE FEM):")
print(f"   - Convergencia: Algebraica O(h²) para elementos lineales")
print(f"   - Elementos recomendados: 40-60 para buena precisión")
print(f"   - Ventaja: Escalable, adaptable, matrices sparse")
print(f"   - Desventaja: Requiere más grados de libertad")

print("\n4. ESTUDIO PARAMÉTRICO DE k:")
for k_val in [k/2, k, 2*k]:
    c_out = solucion_analitica(np.array([L]), D, U, k_val, L, c_inlet)[0]
    conv = (c_inlet - c_out)/c_inlet * 100
    print(f"   k = {k_val:.2f} 1/hr: c(L) = {c_out:.2f} mol/L, Conversión = {conv:.1f}%")

print("\n5. CONCLUSIONES:")
print("   - Ambos métodos convergen correctamente a la solución analítica")
print("   - Legendre es más eficiente para este problema 1D suave")
print("   - FEM sería preferible para geometrías complejas o 2D/3D")
print("   - Mayor k requiere mayor refinamiento numérico")
print("="*70)